# Detección y Manejo de Outliers (Valores Atípicos)

## 1. Configuración Inicial
Importamos las librerías necesarias.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración visual
sns.set_theme(style="whitegrid")

## 2. Caso 1: Propinas en Restaurante (`Tips`)
Analizaremos la variable **`total_bill`** (Cuenta Total).
¿Hay cuentas inusualmente altas que distorsionan nuestro análisis promedio?

In [ ]:
# Cargar dataset
df_tips = sns.load_dataset('tips')

# Visualización Inicial
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histograma
sns.histplot(df_tips['total_bill'], kde=True, ax=axes[0], color='teal')
axes[0].set_title('Distribución Original de Cuentas')

# Boxplot (Los puntos fuera de los "bigotes" son los outliers)
sns.boxplot(x=df_tips['total_bill'], ax=axes[1], color='lightgreen')
axes[1].set_title('Boxplot de Cuentas (Detectando Outliers)')

plt.show()

In [ ]:
# 1. Calcular Q1 (25%) y Q3 (75%)
Q1 = df_tips['total_bill'].quantile(0.25)
Q3 = df_tips['total_bill'].quantile(0.75)
IQR = Q3 - Q1

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")

# 2. Definir límites (Regla del 1.5 * IQR)
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

print(f"Límite para ser outlier: > {limite_superior:.2f} o < {limite_inferior:.2f}")

# 3. Identificar cuántos outliers tenemos
outliers_tips = df_tips[(df_tips['total_bill'] < limite_inferior) | (df_tips['total_bill'] > limite_superior)]
print(f"Número de outliers detectados: {len(outliers_tips)}")
display(outliers_tips.sort_values('total_bill').head())

### Análisis y Decisión
* **Causa:** Los outliers son cuentas superiores a ~$40. Es probable que sean mesas grandes o celebraciones.
* **Decisión:** **NO eliminarlos**. Son datos válidos y representan un segmento de clientes "VIP" o grupos grandes que son rentables para el negocio. Eliminarlos ocultaría información sobre las mejores ventas.

## 3. Caso 2: Tarifas del Titanic (`Fare`)
Analizaremos el precio del boleto. Sabemos que hubo una enorme desigualdad social en el barco.

In [ ]:
# Cargar dataset y limpiar nulos en Fare
df_titanic = sns.load_dataset('titanic')
df_titanic = df_titanic.dropna(subset=['fare'])

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

sns.histplot(df_titanic['fare'], kde=True, ax=axes[0], color='salmon')
axes[0].set_title('Distribución de Tarifas (Con Outliers)')

sns.boxplot(x=df_titanic['fare'], ax=axes[1], color='orange')
axes[1].set_title('Boxplot de Tarifas')

plt.show()

In [ ]:
# Cálculo de límites
Q1 = df_titanic['fare'].quantile(0.25)
Q3 = df_titanic['fare'].quantile(0.75)
IQR = Q3 - Q1

limite_sup_fare = Q3 + 1.5 * IQR

print(f"Cualquier tarifa mayor a ${limite_sup_fare:.2f} es un outlier.")

# Filtrado (Eliminación de outliers para ver cómo cambia la gráfica)
df_sin_outliers = df_titanic[df_titanic['fare'] <= limite_sup_fare]

print(f"Registros originales: {len(df_titanic)}")
print(f"Registros tras limpieza: {len(df_sin_outliers)}")
print(f"Outliers eliminados: {len(df_titanic) - len(df_sin_outliers)}")

### Comparación: Con vs Sin Outliers
Al eliminar los precios excesivamente altos (los ultra-ricos de primera clase), podemos ver mejor la distribución de lo que pagó la "gente normal".

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Antes
sns.histplot(df_titanic['fare'], ax=axes[0], color='gray', alpha=0.5)
axes[0].set_title('Antes: Con Outliers (Escala distorsionada)')

# Después
sns.histplot(df_sin_outliers['fare'], ax=axes[1], color='salmon')
axes[1].set_title('Después: Sin Outliers (Distribución real de la mayoría)')

plt.show()

### Análisis y Decisión
* **Causa:** Los outliers ($512!) corresponden a suites de lujo de primera clase.
* **Decisión:** Depende del objetivo.
    * Si queremos estudiar el comportamiento del pasajero promedio, **eliminarlos** ayuda a ver mejor la realidad del 90% del barco (como se ve en la gráfica derecha).
    * Si estudiamos ingresos totales, debemos mantenerlos.